In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re
import string
from nltk import sent_tokenize

!pip install contractions
import contractions

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 283 kB 34.6 MB/s 
     |████████████████████████████████| 321 kB 67.1 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85451 sha256=0335116ae40d7a9657c5df8ad9c46258a0d5210bf386c9b180f9928ae38798ec
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
stopwords.append('th')
stopwords.append('st')
stopwords.append('nd')
stopwords.append('rd')
len(stopwords)

183

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [5]:
%ls

 amazonreviews.tsv   glove.6B.200d.txt         IMG_20210510_102957-min.jpg
 bricks.jpg          glove.6B.300d.txt         iris.csv.gsheet
 circle.png          glove.6B.50d.txt         'LEARNING MANAGEMENT SYSTEM.pptx'
 Classroom/          glove.6B.zip              nlp_1_train.csv
'Colab Notebooks'/   glove.6B.zip.1            tennis_articles.csv
 Demo.txt            historic_articles.csv
 glove.6B.100d.txt   IMG_20201203_142606.jpg


In [6]:
# Reading csv file
news = pd.read_csv('tennis_articles.csv')

## Analysing data

In [7]:
# Analysing the data
news.head()

,article_id,article_title,article_text,source
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) — Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [8]:
news.tail()

,article_id,article_title,article_text,source
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Rafael Nadal: World No 1 ARRIVES for Paris Mas...,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"TENNIS.COM PODCAST: POINT DEFENSE, RANKING DRO...","Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Tennis journalist’s heartbreaking insight on T...,I PLAYED golf last week with Todd Reid. He pic...,https://www.foxsports.com.au/tennis/tennis-jou...


In [9]:
news.shape

(8, 4)

In [10]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   article_id     8 non-null      int64 
 1   article_title  8 non-null      object
 2   article_text   8 non-null      object
 3   source         8 non-null      object
dtypes: int64(1), object(3)
memory usage: 384.0+ bytes


In [11]:
news.describe()

,article_id
count,8.00000
mean,4.50000
std,2.44949
min,1.00000
25%,2.75000
50%,4.50000
75%,6.25000
max,8.00000


In [12]:
# All the columns in the dataset
news.columns

Index(['article_id', 'article_title', 'article_text', 'source'], dtype='object')

In [13]:
# Checking the news source
news['source'].value_counts()

http://www.tennis.com/pro-game/2018/10/copil-stuns-5th-ranked-zverev-to-reach-swiss-indoors-final/77721/                                                                                1
https://www.express.co.uk/sport/tennis/1037119/Rafael-Nadal-World-No-1-Paris-Masters-Federer-Djokovic                                                                                   1
http://www.tennis.com/pro-game/2018/10/nishikori-beats-kukushkin-in-vienna-for-3rd-final-of-season/77719/                                                                               1
http://www.tennis.com/pro-game/2018/10/tenniscom-podcast-irina-falconi-jack-sock-rafael-nadal-singapore/77698/                                                                          1
https://www.express.co.uk/sport/tennis/1036101/Roger-Federer-Swiss-Indoors-Jan-Lennard-Struff-Lindsay-Davenport                                                                         1
https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-

##Analyzing the news articles

In [14]:
# Analysing the article text.
news['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net. So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same

In [15]:
news['article_text'][7]

'I PLAYED golf last week with Todd Reid. He picked me up at 5.30am, half an hour early because he couldn’t sleep. Or hadn’t slept, to be specific. Not because he’d been out on a bender or anything — those days were in the past. The former Wimbledon junior champion was full of hope, excited about getting his life back together after a troubled few years and a touch-and-go battle with pancreatitis. “I’m pleased with that,” he said after grinding out an eight-over-par front nine at the not-so-royal Northbridge club in Sydney and smashing down an egg- and-bacon roll at the halfway house. To most players of his rare sporting gifts, such a modest return would be unacceptable. To Reid the 15-marker, just being up and at ‘em was enough; a few bogeys and one well-made par — broomstick putter and all — vindication for his recent decision to renew his membership at nearby Bankstown. Exhausted after spending half his round deep in the bushes searching for my ball, as well as those of two other gol

## Tokenizing the text into sentences

In [16]:
news.head()

,article_id,article_title,article_text,source
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) — Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [17]:
news['sen_token'] = news['article_text'].apply(lambda x: sent_tokenize(x))

In [18]:
news.head()

,article_id,article_title,article_text,source,sen_token
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...,[Maria Sharapova has basically no friends as t...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) — Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...,"[BASEL, Switzerland (AP) — Roger Federer advan..."
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...,[Roger Federer has revealed that organisers of...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...,[Kei Nishikori will try to end his long losing...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...,"[Federer, 37, first broke through on tour over..."


In [19]:
# Checking
news['sen_token'][0]

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.",
 "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl.",
 "I say my hellos, but I'm not sending any players flowers as well.",
 "Uhm, I'm not really friendly or close to many players.",
 "I have not a lot of friends away from the courts.'",
 'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
 "Is it different on the men's tour than the women's tour?",
 "'No, not at

In [20]:
news['sen_token'][2]

['Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition.',
 'Speaking at the Swiss Indoors tournament where he will play in Sunday’s final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.',
 '“They only left me three days to decide,” Federer said.',
 '“I didn’t to have time to consult with all the people I had to consult.',
 '“I could not make a decision in that time, so I told them to do what they wanted.” The 20-time Grand Slam champion has voiced doubts about the wisdom of the one-week format to be introduced by organisers Kosmos, who have promised the International Tennis Federation up to $3 billion in prize money over the next quarter-century.',
 'The competition is set to feature 18 countries in the November 18-24 finals in Madrid next year, and will repl

## Text cleaning Part 1

In [21]:
# 1. Expanding contractions
def fix_contraction(text):
  fct = [contractions.fix(s) for s in text]
  return fct

news['clean_text'] = news['sen_token'].apply(lambda x: fix_contraction(x))

In [22]:
news.head() # Everything is expanded
news['clean_text'][1]

['BASEL, Switzerland (AP) — Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday.',
 'Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday.',
 'Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1.',
 "He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand.",
 'He clinched on his fourth chance when Medvedev netted from the baseline.',
 'Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal.',
 'The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent.',
 'Copil has two after also beating No.',
 '6 Marin Cilic in the second round.',
 'Copil fired 26 aces past Zverev and never dropped s

In [23]:
# 2. To lower case
def lower_case(text):
  lr =  [s.lower() for s in text]
  return lr

news['clean_text'] = news['clean_text'].apply(lambda x: lower_case(x))

In [24]:
news['clean_text']
news['clean_text'][1]

['basel, switzerland (ap) — roger federer advanced to the 14th swiss indoors final of his career by beating seventh-seeded daniil medvedev 6-1, 6-4 on saturday.',
 'seeking a ninth title at his hometown event, and a 99th overall, federer will play 93th-ranked marius copil on sunday.',
 'federer dominated the 20th-ranked medvedev and had his first match-point chance to break serve again at 5-1.',
 "he then dropped his serve to love, and let another match point slip in medvedev's next service game by netting a backhand.",
 'he clinched on his fourth chance when medvedev netted from the baseline.',
 'copil upset expectations of a federer final against alexander zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked german in the earlier semifinal.',
 'the romanian aims for a first title after arriving at basel without a career win over a top-10 opponent.',
 'copil has two after also beating no.',
 '6 marin cilic in the second round.',
 'copil fired 26 aces past zverev and never dropped s

In [25]:
news.head(2)

,article_id,article_title,article_text,source,sen_token,clean_text
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...,[Maria Sharapova has basically no friends as t...,[maria sharapova has basically no friends as t...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) — Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...,"[BASEL, Switzerland (AP) — Roger Federer advan...","[basel, switzerland (ap) — roger federer advan..."


## Text cleaning Part 2

In [26]:
# 1. Removing punctuation and special characters
def remove_special_char(text):
  rsc = [re.sub(r'\W+', " ", s) for s in text]
  return rsc 

news['clean_text'] = news['clean_text'].apply(lambda x: remove_special_char(x))

In [27]:
news['clean_text'][1]

['basel switzerland ap roger federer advanced to the 14th swiss indoors final of his career by beating seventh seeded daniil medvedev 6 1 6 4 on saturday ',
 'seeking a ninth title at his hometown event and a 99th overall federer will play 93th ranked marius copil on sunday ',
 'federer dominated the 20th ranked medvedev and had his first match point chance to break serve again at 5 1 ',
 'he then dropped his serve to love and let another match point slip in medvedev s next service game by netting a backhand ',
 'he clinched on his fourth chance when medvedev netted from the baseline ',
 'copil upset expectations of a federer final against alexander zverev in a 6 3 6 7 6 6 4 win over the fifth ranked german in the earlier semifinal ',
 'the romanian aims for a first title after arriving at basel without a career win over a top 10 opponent ',
 'copil has two after also beating no ',
 '6 marin cilic in the second round ',
 'copil fired 26 aces past zverev and never dropped serve clinchin

In [28]:
# 2. Removing numbers
def remove_numbers(text):
  numb = [re.sub(r'\d+\s{1}', "", s) for s in text]
  return numb

news['clean_text'] = news['clean_text'].apply(lambda x: remove_numbers(x))

In [29]:
news['clean_text'][1]

['basel switzerland ap roger federer advanced to the 14th swiss indoors final of his career by beating seventh seeded daniil medvedev on saturday ',
 'seeking a ninth title at his hometown event and a 99th overall federer will play 93th ranked marius copil on sunday ',
 'federer dominated the 20th ranked medvedev and had his first match point chance to break serve again at ',
 'he then dropped his serve to love and let another match point slip in medvedev s next service game by netting a backhand ',
 'he clinched on his fourth chance when medvedev netted from the baseline ',
 'copil upset expectations of a federer final against alexander zverev in a win over the fifth ranked german in the earlier semifinal ',
 'the romanian aims for a first title after arriving at basel without a career win over a top opponent ',
 'copil has two after also beating no ',
 'marin cilic in the second round ',
 'copil fired aces past zverev and never dropped serve clinching after hours with a forehand voll

In [30]:
# 3. Removing remaning digits
def remove_whitespace(text):
  rews = [re.sub(r'\d+', "", s) for s in text]
  return rews

news['clean_text'] = news['clean_text'].apply(lambda x: remove_whitespace(x))

In [31]:
news['clean_text'][1]

['basel switzerland ap roger federer advanced to the th swiss indoors final of his career by beating seventh seeded daniil medvedev on saturday ',
 'seeking a ninth title at his hometown event and a th overall federer will play th ranked marius copil on sunday ',
 'federer dominated the th ranked medvedev and had his first match point chance to break serve again at ',
 'he then dropped his serve to love and let another match point slip in medvedev s next service game by netting a backhand ',
 'he clinched on his fourth chance when medvedev netted from the baseline ',
 'copil upset expectations of a federer final against alexander zverev in a win over the fifth ranked german in the earlier semifinal ',
 'the romanian aims for a first title after arriving at basel without a career win over a top opponent ',
 'copil has two after also beating no ',
 'marin cilic in the second round ',
 'copil fired aces past zverev and never dropped serve clinching after hours with a forehand volley winne

In [32]:
news['clean_text'][2]

['roger federer has revealed that organisers of the re launched and condensed davis cup gave him three days to decide if he would commit to the controversial competition ',
 'speaking at the swiss indoors tournament where he will play in sunday s final against romanian qualifier marius copil the world number three said that given the impossibly short time frame to make a decision he opted out of any commitment ',
 ' they only left me three days to decide federer said ',
 ' i did not to have time to consult with all the people i had to consult ',
 ' i could not make a decision in that time so i told them to do what they wanted the time grand slam champion has voiced doubts about the wisdom of the one week format to be introduced by organisers kosmos who have promised the international tennis federation up to billion in prize money over the next quarter century ',
 'the competition is set to feature countries in the november finals in madrid next year and will replace the classic home an

##Removing STOP words

In [33]:
# function to remove stop words
def remove_stopwords(text):
  resw = " ".join([i for i in text if i not in stopwords]) 
  return resw

news['clean_text'] = news['clean_text'].apply(lambda x: [remove_stopwords(s.split()) for s in x])

In [34]:
news['clean_text'][2]

['roger federer revealed organisers launched condensed davis cup gave three days decide would commit controversial competition',
 'speaking swiss indoors tournament play sunday final romanian qualifier marius copil world number three said given impossibly short time frame make decision opted commitment',
 'left three days decide federer said',
 'time consult people consult',
 'could make decision time told wanted time grand slam champion voiced doubts wisdom one week format introduced organisers kosmos promised international tennis federation billion prize money next quarter century',
 'competition set feature countries november finals madrid next year replace classic home away ties played four times per year decades',
 'kosmos headed barcelona footballer gerard pique hoping fellow spaniard rafael nadal play upcoming event',
 'novak djokovic said give precedence atp intended launch defunct world team cup january various australian venues',
 'major players feel big event late november c

In [35]:
news

,article_id,article_title,article_text,source,sen_token,clean_text
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...,[Maria Sharapova has basically no friends as t...,[maria sharapova basically friends tennis play...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) — Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...,"[BASEL, Switzerland (AP) — Roger Federer advan...",[basel switzerland ap roger federer advanced s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...,[Roger Federer has revealed that organisers of...,[roger federer revealed organisers launched co...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...,[Kei Nishikori will try to end his long losing...,[kei nishikori try end long losing streak atp ...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...,"[Federer, 37, first broke through on tour over...",[federer first broke tour two decades ago sinc...
5,6,Rafael Nadal: World No 1 ARRIVES for Paris Mas...,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...,[Nadal has not played tennis since he was forc...,[nadal played tennis since forced retire us op...
6,7,"TENNIS.COM PODCAST: POINT DEFENSE, RANKING DRO...","Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...,"[Tennis giveth, and tennis taketh away., The e...","[tennis giveth tennis taketh away, end season ..."
7,8,Tennis journalist’s heartbreaking insight on T...,I PLAYED golf last week with Todd Reid. He pic...,https://www.foxsports.com.au/tennis/tennis-jou...,"[I PLAYED golf last week with Todd Reid., He p...","[played golf last week todd reid, picked half ..."


## Word Embeddings

### Downloading word embeddings
Wll be using Glove embeddings

In [36]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-08-15 14:48:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-08-15 14:48:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-08-15 14:48:01--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [37]:
# Extracting word embeddigs
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding = 'utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  word_embeddings[word] = coefs
f.close()

In [38]:
len(word_embeddings)

400000

###Creating vectors for sentences
We will create the vectors of size 100

In [39]:
def vectorizing(article):
  article_vector = []
  for sen in article:
    if len(sen) != 0:
      v = sum([word_embeddings.get(w, np.zeros((100,))) for w in sen.split()]) / (len(sen.split()) + 0.001)
    else:
      v = np.zeros((100,))
    article_vector.append(v)
  return article_vector

a = news['clean_text'].apply(lambda x: vectorizing(x))
vect_df = pd.DataFrame(a)
vect_df

,clean_text
0,"[[0.05148256, 0.11054468, 0.6949994, 0.1891681..."
1,"[[0.10586457, -0.104832724, 0.5098503, 0.25731..."
2,"[[-0.014404414, 0.00031916526, 0.118812695, -0..."
3,"[[0.04519946, -0.06464375, 0.5035452, -0.16035..."
4,"[[0.21534438, 0.18089992, 0.2559875, 0.0692366..."
5,"[[0.03523205, -0.03176445, 0.36155656, -0.1280..."
6,"[[-0.115314946, -0.006283154, 0.595066, -0.160..."
7,"[[-0.15236476, 0.05248125, 0.46090904, -0.5684..."


In [40]:
vect_df['clean_text'][0]

[array([ 5.14825583e-02,  1.10544682e-01,  6.94999397e-01,  1.89168096e-01,
        -9.58077684e-02,  3.20288986e-01,  2.70662010e-01,  5.42440832e-01,
        -3.05938005e-01, -1.56364068e-01,  3.70127618e-01,  8.09492469e-02,
         8.41393881e-03,  2.47571543e-01, -3.69342804e-01, -7.61044994e-02,
         8.08582604e-02,  2.30643645e-01, -2.70402402e-01,  5.13828397e-01,
        -6.12548441e-02,  3.87900352e-01,  1.03121363e-01,  7.72494674e-01,
         2.59960234e-01, -7.96069205e-02,  1.42143592e-01, -9.62644577e-01,
         7.54904330e-01,  6.03260659e-02, -4.58570123e-01,  2.36780301e-01,
         2.29152635e-01, -1.56453326e-01,  3.97632688e-01, -2.32720934e-02,
        -5.05520999e-01,  4.13252831e-01, -2.85759270e-01, -1.35231465e-01,
        -1.37098104e-01, -1.48972601e-01,  3.37537557e-01, -3.49540442e-01,
         1.53484434e-01, -2.33341649e-01, -1.98460802e-01, -1.27821520e-01,
         5.08063912e-01, -3.68636876e-01, -2.28472307e-01, -3.15306723e-01,
         1.3

## Preparing a similarity matrix
we will find the cosine similarity of the sentences

In [41]:
vect_df['clean_text']

0    [[0.05148256, 0.11054468, 0.6949994, 0.1891681...
1    [[0.10586457, -0.104832724, 0.5098503, 0.25731...
2    [[-0.014404414, 0.00031916526, 0.118812695, -0...
3    [[0.04519946, -0.06464375, 0.5035452, -0.16035...
4    [[0.21534438, 0.18089992, 0.2559875, 0.0692366...
5    [[0.03523205, -0.03176445, 0.36155656, -0.1280...
6    [[-0.115314946, -0.006283154, 0.595066, -0.160...
7    [[-0.15236476, 0.05248125, 0.46090904, -0.5684...
Name: clean_text, dtype: object

In [42]:
len(news['clean_text'][1])

12

Let’s first define a zero matrix of dimensions (n * n).  We will initialize this matrix with cosine similarity scores of the sentences of an article. Here, n is the number of sentences in an article.

In [43]:
vect_df['similarity_matrix'] = news['sen_token'].apply(lambda x: np.zeros([len(x), len(x)]))

In [44]:
vect_df

,clean_text,similarity_matrix
0,"[[0.05148256, 0.11054468, 0.6949994, 0.1891681...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,"[[0.10586457, -0.104832724, 0.5098503, 0.25731...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,"[[-0.014404414, 0.00031916526, 0.118812695, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,"[[0.04519946, -0.06464375, 0.5035452, -0.16035...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"[[0.21534438, 0.18089992, 0.2559875, 0.0692366...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,"[[0.03523205, -0.03176445, 0.36155656, -0.1280...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,"[[-0.115314946, -0.006283154, 0.595066, -0.160...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,"[[-0.15236476, 0.05248125, 0.46090904, -0.5684...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [45]:
vect_df['sents'] = news['sen_token'].values

In [46]:
vect_df

,clean_text,similarity_matrix,sents
0,"[[0.05148256, 0.11054468, 0.6949994, 0.1891681...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[Maria Sharapova has basically no friends as t...
1,"[[0.10586457, -0.104832724, 0.5098503, 0.25731...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[BASEL, Switzerland (AP) — Roger Federer advan..."
2,"[[-0.014404414, 0.00031916526, 0.118812695, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[Roger Federer has revealed that organisers of...
3,"[[0.04519946, -0.06464375, 0.5035452, -0.16035...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[Kei Nishikori will try to end his long losing...
4,"[[0.21534438, 0.18089992, 0.2559875, 0.0692366...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[Federer, 37, first broke through on tour over..."
5,"[[0.03523205, -0.03176445, 0.36155656, -0.1280...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",[Nadal has not played tennis since he was forc...
6,"[[-0.115314946, -0.006283154, 0.595066, -0.160...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[Tennis giveth, and tennis taketh away., The e..."
7,"[[-0.15236476, 0.05248125, 0.46090904, -0.5684...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[I PLAYED golf last week with Todd Reid., He p..."


In [49]:
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
# Assgning the matrix with similarity scores
for k in vect_df.iloc:
  x = k.name
  for i in range(len(vect_df['sents'][x])):
    for j in range(len(vect_df['sents'][x])):
      if i != j:
        vect_df['similarity_matrix'][x][i][j] = cosine_similarity(vect_df['clean_text'][x][i].reshape(1,100), vect_df['clean_text'][x][j].reshape(1, 100))[0,0]

In [51]:
vect_df['similarity_matrix']

0    [[0.0, 0.6437833905220032, 0.5915699601173401,...
1    [[0.0, 0.8939819931983948, 0.8638535141944885,...
2    [[0.0, 0.9248484969139099, 0.9090324640274048,...
3    [[0.0, 0.7852268815040588, 0.8482545614242554,...
4    [[0.0, 0.810055423540263, 0.9082390666007996, ...
5    [[0.0, 0.8968858122825623, 0.8748721480369568,...
6    [[0.0, 0.47489437460899353, 0.4500268101692199...
7    [[0.0, 0.7607073187828064, 0.438073068857193, ...
Name: similarity_matrix, dtype: object

## Applying page rank algorithm



In [52]:
# converting this similarity matrix to a graph. 
# The nodes of this graph will represent the tokenized sentences.
# edges will represent the similarity score b/w sentences.

import networkx as nx

In [53]:
vect_df['nx_graph'] = vect_df['similarity_matrix'].apply(lambda x: nx.from_numpy_array(x))

In [54]:
vect_df['scores'] = vect_df['nx_graph'].apply(lambda x: nx.pagerank(x))

In [55]:
vect_df['scores']

0    {0: 0.049271663661586376, 1: 0.056260332812872...
1    {0: 0.08347728351009916, 1: 0.0857253951885348...
2    {0: 0.07091610358816944, 1: 0.0716801831901816...
3    {0: 0.08109867192976133, 1: 0.0729409613094228...
4    {0: 0.05944143827301597, 1: 0.0531357506715829...
5    {0: 0.08350049169380515, 1: 0.0841110667571069...
6    {0: 0.06800464376414277, 1: 0.1059924009444186...
7    {0: 0.031776818539783785, 1: 0.031917881509573...
Name: scores, dtype: object

In [56]:
vect_df

,clean_text,similarity_matrix,sents,nx_graph,scores
0,"[[0.05148256, 0.11054468, 0.6949994, 0.1891681...","[[0.0, 0.6437833905220032, 0.5915699601173401,...",[Maria Sharapova has basically no friends as t...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.049271663661586376, 1: 0.056260332812872..."
1,"[[0.10586457, -0.104832724, 0.5098503, 0.25731...","[[0.0, 0.8939819931983948, 0.8638535141944885,...","[BASEL, Switzerland (AP) — Roger Federer advan...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","{0: 0.08347728351009916, 1: 0.0857253951885348..."
2,"[[-0.014404414, 0.00031916526, 0.118812695, -0...","[[0.0, 0.9248484969139099, 0.9090324640274048,...",[Roger Federer has revealed that organisers of...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.07091610358816944, 1: 0.0716801831901816..."
3,"[[0.04519946, -0.06464375, 0.5035452, -0.16035...","[[0.0, 0.7852268815040588, 0.8482545614242554,...",[Kei Nishikori will try to end his long losing...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","{0: 0.08109867192976133, 1: 0.0729409613094228..."
4,"[[0.21534438, 0.18089992, 0.2559875, 0.0692366...","[[0.0, 0.810055423540263, 0.9082390666007996, ...","[Federer, 37, first broke through on tour over...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.05944143827301597, 1: 0.0531357506715829..."
5,"[[0.03523205, -0.03176445, 0.36155656, -0.1280...","[[0.0, 0.8968858122825623, 0.8748721480369568,...",[Nadal has not played tennis since he was forc...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","{0: 0.08350049169380515, 1: 0.0841110667571069..."
6,"[[-0.115314946, -0.006283154, 0.595066, -0.160...","[[0.0, 0.47489437460899353, 0.4500268101692199...","[Tennis giveth, and tennis taketh away., The e...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)","{0: 0.06800464376414277, 1: 0.1059924009444186..."
7,"[[-0.15236476, 0.05248125, 0.46090904, -0.5684...","[[0.0, 0.7607073187828064, 0.438073068857193, ...","[I PLAYED golf last week with Todd Reid., He p...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.031776818539783785, 1: 0.031917881509573..."


## Summary Extraction

In [57]:
vect_df['ranked_sents'] = vect_df['sents'].apply(lambda x: x)

In [58]:
vect_df

,clean_text,similarity_matrix,sents,nx_graph,scores,ranked_sents
0,"[[0.05148256, 0.11054468, 0.6949994, 0.1891681...","[[0.0, 0.6437833905220032, 0.5915699601173401,...",[Maria Sharapova has basically no friends as t...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.049271663661586376, 1: 0.056260332812872...",[Maria Sharapova has basically no friends as t...
1,"[[0.10586457, -0.104832724, 0.5098503, 0.25731...","[[0.0, 0.8939819931983948, 0.8638535141944885,...","[BASEL, Switzerland (AP) — Roger Federer advan...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","{0: 0.08347728351009916, 1: 0.0857253951885348...","[BASEL, Switzerland (AP) — Roger Federer advan..."
2,"[[-0.014404414, 0.00031916526, 0.118812695, -0...","[[0.0, 0.9248484969139099, 0.9090324640274048,...",[Roger Federer has revealed that organisers of...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.07091610358816944, 1: 0.0716801831901816...",[Roger Federer has revealed that organisers of...
3,"[[0.04519946, -0.06464375, 0.5035452, -0.16035...","[[0.0, 0.7852268815040588, 0.8482545614242554,...",[Kei Nishikori will try to end his long losing...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","{0: 0.08109867192976133, 1: 0.0729409613094228...",[Kei Nishikori will try to end his long losing...
4,"[[0.21534438, 0.18089992, 0.2559875, 0.0692366...","[[0.0, 0.810055423540263, 0.9082390666007996, ...","[Federer, 37, first broke through on tour over...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.05944143827301597, 1: 0.0531357506715829...","[Federer, 37, first broke through on tour over..."
5,"[[0.03523205, -0.03176445, 0.36155656, -0.1280...","[[0.0, 0.8968858122825623, 0.8748721480369568,...",[Nadal has not played tennis since he was forc...,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","{0: 0.08350049169380515, 1: 0.0841110667571069...",[Nadal has not played tennis since he was forc...
6,"[[-0.115314946, -0.006283154, 0.595066, -0.160...","[[0.0, 0.47489437460899353, 0.4500268101692199...","[Tennis giveth, and tennis taketh away., The e...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)","{0: 0.06800464376414277, 1: 0.1059924009444186...","[Tennis giveth, and tennis taketh away., The e..."
7,"[[-0.15236476, 0.05248125, 0.46090904, -0.5684...","[[0.0, 0.7607073187828064, 0.438073068857193, ...","[I PLAYED golf last week with Todd Reid., He p...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{0: 0.031776818539783785, 1: 0.031917881509573...","[I PLAYED golf last week with Todd Reid., He p..."


In [59]:
# Sorting sentences based on rank
for k in vect_df.iloc:
  x = k.name
  vect_df['ranked_sents'][x] = sorted(((vect_df['scores'][x][i],s) for i,s in enumerate(vect_df['sents'][x])), reverse = True)

In [60]:
vect_df['ranked_sents'][0]

[(0.059013484475260516,
  "I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players."),
 (0.05775785287248156,
  "I think everyone just thinks because we're tennis players we should be the greatest of friends."),
 (0.05746240254304827,
  'When she said she is not really close to a lot of players, is that something strategic that she is doing?'),
 (0.05710511681608242,
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net."),
 (0.056555629604371035,
  "Uhm, I'm not really friendly or close to many players."),
 (0.05638733078559284,
  "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."),
 (0.05626033281287299,
  "The Russian player 

## Extracting top 40% sentences for the summary

In [61]:
def summary(text):
  l = int(len(text) * 2 / 5)
  print('\nSumary is :- \n')
  for i in range(l):
    print(text[i][1])
  
vect_df['ranked_sents'].apply(lambda x: summary(x))


Sumary is :- 

I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players.
I think everyone just thinks because we're tennis players we should be the greatest of friends.
When she said she is not really close to a lot of players, is that something strategic that she is doing?
When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.
Uhm, I'm not really friendly or close to many players.
So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.

Sumary is :- 

Federer dominated the 20th-ranked Medvedev and

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
Name: ranked_sents, dtype: object